In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import cvxpy as cp
import plotly.graph_objects as go


In [8]:
tickers = [
    "SPY",   # SPDR S&P 500 ETF Trust
    "AAPL",  # Apple Inc.
    "WFC",   # Wells Fargo & Co
    "XOM",   # Exxon Mobil Corp
    "JNJ",   # Johnson & Johnson
    "SBUX"   # Starbucks Corp
]

# Diccionario con tickers, nombres de empresas y sectores
empresas_dict = {
    "SPY": {"nombre": "S&P 500 ETF", "sector": "Índice", "alcance": "Global"},
    "AAPL": {"nombre": "Apple Inc.", "sector": "Tecnología / Electrónica de Consumo", "alcance": "Global"},
    "WFC": {"nombre": "Wells Fargo & Co", "sector": "Servicios Financieros / Bancario", "alcance": "Local"},
    "XOM": {"nombre": "Exxon Mobil Corp", "sector": "Petróleo y Gas", "alcance": "Global"},
    "JNJ": {"nombre": "Johnson & Johnson", "sector": "Farmacéutica y Productos de Consumo", "alcance": "Global/Local"},
    "SBUX": {"nombre": "Starbucks Corp", "sector": "Alimentación y Bebidas / Servicio de Cafeterías", "alcance": "Local"}
}


In [9]:
precios = yf.download(tickers, start="2019-01-01", end="2020-12-31")["Adj Close"] # Precios ajustados al cierre
precios = precios.dropna() # Eliminar filas con NA's

# Calcular rendimientos logarítmicos
rendimientos = np.log(precios / precios.shift(1)).dropna()


[*********************100%%**********************]  6 of 6 completed


In [6]:
# Calculate betas
betas = {}
for column in rendimientos.columns:
    cov = np.cov(rendimientos['SPY'], rendimientos[column])[0, 1]
    var = np.var(rendimientos['SPY'])
    beta = cov / var
    betas[column] = beta

# Print betas
for asset, beta in betas.items():
    print(f"Beta of {asset} with respect to SPY: {beta}")

Beta of AAPL with respect to SPY: 1.206456322945742
Beta of JNJ with respect to SPY: 0.6647198042550472
Beta of SBUX with respect to SPY: 1.1161350551619378
Beta of SPY with respect to SPY: 1.00199203187251
Beta of WFC with respect to SPY: 1.3616428835255525
Beta of XOM with respect to SPY: 1.1229811944254695


In [2]:
companies = pd.read_csv("../data/index-components-01-11-2024.csv",)
companies = companies.iloc[:-1,:] # Eliminar última fila

companies_dict = {companies["Symbol"][i]: {"name": companies["Name"][i]} for i in range(len(companies))}

In [18]:
precios = yf.download(list(companies_dict.keys()), start="2019-01-01", end="2023-12-31")["Adj Close"] # Precios ajustados al cierre

[*********************100%%**********************]  503 of 503 completed


2 Failed downloads:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2019-01-01 -> 2023-12-31)')


In [9]:
precios = pd.read_csv("../data/sp500_precios.csv", index_col=0, parse_dates=True)
precios = precios.dropna(axis=1) # Eliminar columnas con NA's
precios.head(5)

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,63.381710,31.963160,37.893341,70.646538,NaN,63.908173,26.190001,131.077469,224.570007,78.138260,...,99.469666,41.850040,53.962513,35.926987,62.568439,83.576164,95.619514,156.240005,35.226196,81.659958
2019-01-03,61.046745,29.581663,34.118874,68.318840,NaN,60.892082,25.780001,126.602242,215.699997,73.418312,...,98.507370,41.685143,53.133984,35.945953,60.423553,81.473938,93.889969,146.880005,35.031338,78.837448
2019-01-04,63.159786,31.530161,35.575378,70.519852,NaN,62.630009,26.389999,131.524994,226.190002,75.200783,...,102.480431,42.093063,55.093021,36.988953,62.972939,83.594421,97.096619,152.970001,36.107277,81.930641
2019-01-07,64.500923,32.425674,35.496204,71.549126,NaN,63.567928,26.330000,131.981812,229.259995,75.673691,...,104.300240,41.910801,55.379513,37.652691,62.041607,83.503029,97.134010,155.289993,36.251289,82.423607
2019-01-08,65.446503,31.904114,36.172874,71.881653,NaN,62.804718,26.430000,135.319580,232.679993,77.519836,...,107.568253,42.396820,55.782177,37.396687,62.624863,83.338516,94.955734,156.330002,36.581703,83.651207


In [13]:
ceros = precios.columns[precios.isna().sum() != 0]
ceros

Index(['ABNB', 'BF.B', 'BRK.B', 'CARR', 'CEG', 'CTVA', 'DOW', 'FOX', 'FOXA',
       'GEHC', 'KVUE', 'OTIS', 'UBER', 'VLTO'],
      dtype='object')

In [14]:
precios.loc[:, ceros].isna().sum()

ABNB      490
BF.B     1258
BRK.B    1258
CARR      305
CEG       768
CTVA       99
DOW        53
FOX        48
FOXA       47
GEHC      997
KVUE     1092
OTIS      305
UBER       89
VLTO     1197
dtype: int64

In [ ]:
retornos = np.log(precios / precios.shift(1)).dropna()